# Anzeige der Temperaturdaten aus der Heizung aus Temperaturen.db

In [ ]:
'''
Datenbankformat: CREATE TABLE Temperaturen('UnixTime', 'AussenTemp', 'VorlaufTemp', 'RuecklaufTemp', 'WohnzimmerTemp', 'Steigung', 'Niveau', 'UPumpe')
sk,09,11,23 Fertig
sk,27,11,23 Kopieren Datei über http fertig
'''

import platform
print(platform.system() + platform.release())
import os, sys, shutil
import ipywidgets as widgets
output = widgets.Output()
from ipywidgets import HBox, VBox, Button, Layout, Label, interact, interactive, fixed, interact_manual
import time
import datetime as dt
import sqlite3
import numpy as np
import warnings
#from dotenv import dotenv_values
from zoneinfo import ZoneInfo
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import iplot
#pio.renderers.default='iframe'
# geht auch in VSCODE:
pio.renderers.default='vscode'
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.plotting.backend = "plotly"
pd.options.mode.chained_assignment = None 

DB_FILENAME = 'Temperaturen.db'
DB_HTTP_PATH='192.168.202.41/grafana/'
DEFAULT_DURATION_DAYS=6
AGE_DIFF_SECONDS = 60*60*24


VARIABLES = {}
with open(".env") as f:
    for line in f:
        name, value = line.split("=")
        VARIABLES[name] = value
#for var in VARIABLES: print(str(var) + ': ' + VARIABLES.get(var))
 
def get_db_file():
    print('Hole Daten ...', end = '')
    cmd_line='curl ' + DB_HTTP_PATH + DB_FILENAME + ' -o ' + DB_FILENAME
    print(cmd_line)
    try:
        os.system(cmd_line)
        print('OK')
        return True
    except:
        print('Kopieren der Daten nicht möglich!')
        return False
    
    '''
    try:
        os.system(cmd_line)
    #config = dotenv_values(".env")
    full_filename= DB_UNC_PATH + DB_FILENAME
    try:
        shutil.copyfile(full_filename, './' + DB_FILENAME)        
        print('Feddisch')
        return True
    except:
        print('Kopieren der Daten nicht möglich!')
        return False
    '''

def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
    
def plot_dfplot(start, dauer, einheit):
    start_object = dt.datetime(start.year, start.month, start.day)
    
    if einheit == 'Tage':
        end_date = start_object + dt.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_date = start_object + dt.timedelta(hours=dauer)
    else:
        end_date = start_object + dt.timedelta(days=dauer)
    #print('Start: ' + str(start) + ' End: ' + str(end_date))
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_date )]
    subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"], grid=True)

def plot_temperaturverlauf(df, start, dauer, show_spreizung, show_heizungs_einstellungen):
    start_object = dt.datetime(start.year, start.month, start.day,  tzinfo=ZoneInfo("Europe/Berlin"))   
    end_object = start_object + dt.timedelta(hours=dauer)
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object )]
    subset = subset[(subset['UnixTime'] <= end_object)]
    #display(subset['UnixTime'].min)
    #display(subset['UnixTime'].max)
    
    subset.set_index(['UnixTime'])
    if show_spreizung: subset['Spreizung'] = subset['VorlaufTemp'] - subset['RuecklaufTemp'] 
    subset = subset.astype({'WohnzimmerTemp':'float64'})
    subset = subset.astype({'AussenTemp':'float64'})    
    fig_title = ""
    try:
        fig_title = "Temperaturverlauf Heizung: " + subset['UnixTime'].min().strftime('%d.%m.%Y') + " - " + subset['UnixTime'].max().strftime('%d.%m.%Y')
    except:
        fig_title = "Temperaturverlauf Heizung"

    kurven= ['VorlaufTemp', 'RuecklaufTemp', 'WohnzimmerTemp', 'AussenTemp']
    einstellungs_kurven= ['Steigung', 'Niveau','UPumpe']
    if show_spreizung: kurven.append('Spreizung')
    if show_heizungs_einstellungen: 
        kurven.extend(einstellungs_kurven)        
    #print(str(kurven))
    fig_verlauf = subset.plot(x='UnixTime',y=kurven, title=fig_title,
                     labels={
                     "UnixTime": "Zeit",
                     "VorlaufTemp": "VorlaufTemperatur",
                     "UPumpe": "Einstellung Umwälzpumpe"
                 },)
    fig_verlauf.update_layout(xaxis_title="Zeit", yaxis_title="Temperaturen in °C", legend_title=None)    
    fig_verlauf.show()
            
        
def set_button_text(b, DB_FILENAME, AGE_DIFF_SECONDS):
    is_too_old, btn_text =get_temperatur_file_status(DB_FILENAME, AGE_DIFF_SECONDS)
    b.description=btn_text
    if is_too_old:
        b.style.button_color = '#90ee90'
    else:
        b.style.button_color = '#FF0000'
    
def prepare_file():
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')
    df_orig = df
    df.set_index(['UnixTime'])
    return df

def get_temperatur_file_status(db_filename, agediff):
    st=os.stat(db_filename)    
    mtime=st.st_mtime
    format_str='%d.%m.%Y %H:%M'
    if time.time() - os.path.getmtime(db_filename) > agediff:
        return (False, 'Temperaturdaten: ' + format(dt.datetime.fromtimestamp(mtime),format_str) )
    else:
        return (True, 'Temperaturdaten: ' + format(dt.datetime.fromtimestamp(mtime),format_str))

    
def display_temperatur_verlauf():
    df = None
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')
        
    df.index = df['UnixTime']

    wstyle = {'description_width': 'initial'}
    opts = [('1 Stunde', 1), ('6 Stunden', 6), ('12 Stunden', 12), ('24 Stunden', 24), ('2 Tage', 48),('3 Tage', 72), ('4 Tage', 96), ('5 Tage', 120), ('6 Tage', 144), ('7 Tage', 168)]
    widget_dauer = widgets.Dropdown(options=opts,value=168,description='Anzeigedauer:', style=wstyle)
    art_der_kurve = ['Temperaturverlauf', 'Spreizung']
    widget_start_picker = widgets.DatePicker(description='Beginn Zeitraum', value = dt.datetime.now().date() - dt.timedelta(days=DEFAULT_DURATION_DAYS-1), style=wstyle)    
    widget_spreizung=widgets.Checkbox(value=False,description='Spreizungskurve anzeigen')
    widget_einstellungen=widgets.Checkbox(value=False,description='Heizungseinstellungen anzeigen')
    
    wdg=widgets.interactive_output(plot_temperaturverlauf, {'df': widgets.fixed(df), 'start': widget_start_picker, 'dauer': widget_dauer, 
                                                            'show_spreizung' : widget_spreizung, 'show_heizungs_einstellungen': widget_einstellungen})
    hbox1 = widgets.HBox([widget_spreizung, widget_einstellungen])
    hbox2 = widgets.HBox([widget_start_picker, widget_dauer])
    ui = widgets.VBox([hbox2, hbox1])
    display(ui,wdg)

def display_kurve(function):
    wstyle = {'description_width': 'initial'}
    widget_start_picker = widgets.DatePicker(description='Von: ', value = dt.datetime.now().date() - dt.timedelta(days=30), style=wstyle)
    widget_end_picker = widgets.DatePicker(description='Bis:', value = dt.datetime.now().date() + dt.timedelta(days=1), style=wstyle)
    widget_label = widgets.Label(value="")

    TIME_FROM = "12:00"
    TIME_TO = "20:00"
    dates = pd.date_range(TIME_FROM, TIME_TO, freq="30min").time
    print(VARIABLES.get('TIME_RANGE_FROM'))
    print(VARIABLES.get('TIME_RANGE_TO'))
    dates = pd.date_range("10:00", "20:00", freq="30min").time

    options = [date.strftime('%H:%M') for date in dates]
    index = (0, len(options)-1)

    start_time_von = "14:00"
    start_time_bis = "17:00"
    timerange_slider = widgets.SelectionRangeSlider(
        value=(start_time_von, start_time_bis),
        continuous_update=False,
        options=options,
        index=index,
        description='Zeitraum',
        orientation='horizontal',
        layout={'width': '300px'}
    )
    
    df = None
    try:    
        df= get_df(DB_FILENAME)
    except SystemExit as _e:
        print(_e)
        sys.exit(-1)

    # https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
    df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')

    #https://stackoverflow.com/questions/46576831/get-typeerror-index-must-be-datetimeindex-when-filtering-dataframe
    df.index = df['UnixTime']
    
    #tk_widget=widgets.interactive_output(plot_heizkurve, {'df': widgets.fixed(df1), 'start': widget_start_picker, 'end': widget_end_picker, 'label': widget_label})
    tk_widget=widgets.interactive_output(function, {'df': widgets.fixed(df), 'startdate': widget_start_picker, 'enddate': widget_end_picker, 'label': widget_label, 'timerange': timerange_slider})
    hbox1 = widgets.HBox([widget_start_picker, widget_end_picker, widget_label, timerange_slider])
    #hbox2 = widgets.HBox([widget_start_picker, widget_dauer, widget_button])
    ui = widgets.VBox([hbox1])
    display(ui,tk_widget)      
    
def plot_heizkennlinie(df, startdate, enddate, label, timerange):
    # Anzeigen Heizkurve: Beziehung zwischen Vorlauftemperatur und Aussentemperatur bei konstanten Heizungseinstellungen
    start_object = dt.datetime(startdate.year, startdate.month, startdate.day,  tzinfo=ZoneInfo("Europe/Berlin"))   
    end_object = dt.datetime(enddate.year, enddate.month, enddate.day,  tzinfo=ZoneInfo("Europe/Berlin"))   
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_object )]
    TIME_FROM= timerange[0]
    TIME_TO= timerange[1]
    df = subset.between_time(TIME_FROM,TIME_TO)
    
    df.to_csv('Daten_Kennlinie.csv', index = False, sep ='\t')

    upumpe_list = df['UPumpe'].unique()
    niveau_list = df['Niveau'].unique()
    steigung_list = df['Steigung'].unique()
    
    vals_tuples = tuple([l1, l2, l3] for l1 in steigung_list for l2 in niveau_list for l3 in upumpe_list)
    heizkurven = {}
    for tup in vals_tuples:
        color = '#FF0000'
        descr = "Steig: {} Niv: {} Pumpe: {}".format(*tup)
        hk = df[(df['Steigung'] == tup[0] ) &  (df['Niveau'] == tup[1] ) & (df['UPumpe'] == tup[2] )].sort_values(by=['AussenTemp'])
        heizkurven[tuple(tup)] = [color, descr, hk]

    #https://stackoverflow.com/questions/64500472/plot-multiple-pandas-dataframes-in-one-graph-python-plotly
    fig_hk = go.Figure()
    the_title="Heizkennlinie: Abhängigkeit Vorlauftemperatur von Aussentemperatur, Messbereich: " + TIME_FROM + ' - ' + TIME_TO
    fig_hk.update_layout(xaxis_title="Aussentemperatur [°C]", yaxis_title="Vorlaufemperatur [°C]", title=the_title)  
    heizkurve_list=[]
    for key, value in heizkurven.items():        
        data= value[2]
        color = value[0]
        MAT = data.groupby(pd.PeriodIndex(data['UnixTime'], freq="D"))['AussenTemp'].mean()
        MVT = data.groupby(pd.PeriodIndex(data['UnixTime'], freq="D"))['VorlaufTemp'].mean()
        heizkurve = pd.merge(MAT, MVT, on='UnixTime').sort_values(by=['AussenTemp'])
        heizkurve_list.append((heizkurve, value[1]))
        fig_hk = fig_hk.add_trace(go.Scatter(x = heizkurve['AussenTemp'], y = heizkurve['VorlaufTemp'], name = value[1]))    
    fig_hk.show()

    print('Wertetabelle:')
    for hk in heizkurve_list:
        print(hk[1])
        print(hk[0])

def plot_wohnzimmerkurve(df, startdate, enddate, label, timerange):
    # Anzeigen Stubenkurve: Beziehung zwischen Vorlauftemperatur und Raumtemperatur bei konstanten Heizungseinstellungen
    start_object = dt.datetime(startdate.year, startdate.month, startdate.day,  tzinfo=ZoneInfo("Europe/Berlin"))   
    end_object = dt.datetime(enddate.year, enddate.month, enddate.day,  tzinfo=ZoneInfo("Europe/Berlin")) 
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_object )]
    TIME_FROM = timerange[0]
    TIME_TO= timerange[1]
    df = subset.between_time(TIME_FROM,TIME_TO)
    
    df.to_csv('Daten_Kennlinie.csv', index = False, sep ='\t')

    upumpe_list = df['UPumpe'].unique()
    niveau_list = df['Niveau'].unique()
    steigung_list = df['Steigung'].unique()
    
    vals_tuples = tuple([l1, l2, l3] for l1 in steigung_list for l2 in niveau_list for l3 in upumpe_list)
    heizkurven = {}
    for tup in vals_tuples:
        #print(tup)
        color = '#FF0000'
        descr = "Steig: {} Niv: {} Pumpe: {}".format(*tup)
        #print(descr)
        hk = df[(df['Steigung'] == tup[0] ) &  (df['Niveau'] == tup[1] ) & (df['UPumpe'] == tup[2] )].sort_values(by=['AussenTemp'])
        heizkurven[tuple(tup)] = [color, descr, hk]

    #https://stackoverflow.com/questions/64500472/plot-multiple-pandas-dataframes-in-one-graph-python-plotly
    fig_hk = go.Figure()
    the_title="Abhängigkeit Wohnzimmertemperatur von der Vorlauftemperatur"
    fig_hk.update_layout(xaxis_title="Vorlaufemperatur [°C]", yaxis_title="Raumtemperatur [°C]", title=the_title)  
    heizkurve_list=[]
    for key, value in heizkurven.items():        
        data= value[2]
        color = value[0]
        MMT = data.groupby(pd.PeriodIndex(data['UnixTime'], freq="D"))['WohnzimmerTemp'].mean()
        MVT = data.groupby(pd.PeriodIndex(data['UnixTime'], freq="D"))['VorlaufTemp'].mean()
        heizkurve = pd.merge(MMT, MVT, on='UnixTime').sort_values(by=['VorlaufTemp'])
        heizkurve_list.append((heizkurve, value[1]))
        fig_hk = fig_hk.add_trace(go.Scatter(x = heizkurve['VorlaufTemp'], y = heizkurve['WohnzimmerTemp'], name = value[1]))    
    fig_hk.show()

    print('Wertetabelle:')
    for hk in heizkurve_list:
        print(hk[1])
        print(hk[0])
        
@output.capture()
def get_file_button_clicked(b):
    with output:
        b.description="Kopiere Datei " + DB_UNC_PATH + DB_FILENAME + ' ...'
        if get_db_file():
            set_button_text(b, DB_FILENAME, AGE_DIFF_SECONDS)
        else:
            b.description="Fehler beim Kopieren der Datei!"
            b.style.button_color = '#FF0000'

def do_display_all(select, btn, text):
    if select == 'Temperaturverlauf':
        display_temperatur_verlauf()
    if select == 'Heizkennlinie':
        display_kurve(plot_heizkennlinie)
    if select == 'Wohnzimmerkurve':
        display_kurve(plot_wohnzimmerkurve)

def display_all():
    df = prepare_file()
    #display(df.iloc[-1])
    a_temp = df['AussenTemp'].iloc[-1]
    v_temp = df['VorlaufTemp'].iloc[-1]
    tup=(a_temp,v_temp)
    #tup = tail

    aktuell = "Aussen: {:2.1f} °C Vorlauf: {:.2f} °C".format(*tup)
    
    widget_choice = widgets.Dropdown(options=['Bitte wählen','Temperaturverlauf', 'Heizkennlinie', 'Wohnzimmerkurve'])
    
    layout_btn = widgets.Layout(width='auto')
    widget_temperature_button = widgets.Button(layout=layout_btn)
    set_button_text(widget_temperature_button, DB_FILENAME, AGE_DIFF_SECONDS)
    widget_temperature_button.on_click(get_file_button_clicked)
    widget_temperature_button.value = "Temperaturverlauf"
    widget_text = widgets.Text(value=aktuell, placeholder='Type something',description='Aktuell:',layout = widgets.Layout(width='450px'))
    da_widget=widgets.interactive_output(do_display_all, {'select': widget_choice, 'btn' : widget_temperature_button, 'text': widget_text})
    hbox1 = widgets.HBox([widget_choice, widget_temperature_button, widget_text])
    #hbox1 = widgets.HBox([widget_temperature_button])
    ui = widgets.VBox([hbox1])
    display(ui,da_widget)      

display_all()